In [55]:
import cv2
import os
import mediapipe as mp
import numpy as np

In [56]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [57]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [58]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                             mp_drawing.DrawingSpec(color=(80,110, 10), thickness=1, circle_radius=1),
                              mp_drawing.DrawingSpec(color=(80,256, 121), thickness=1, circle_radius=1)
                             )
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22, 10), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(80,44, 121), thickness=2, circle_radius=2)
                             )
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(121, 44, 250), thickness=2, circle_radius=2)
                             )
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245,117, 66), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(245,66, 230), thickness=2, circle_radius=2),
                             )
    

In [18]:
cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)  as holistic:
    while cap.isOpened():

        ret, frame = cap.read()
        
        # make detection
        image, results = mediapipe_detection(frame, holistic)
        

        draw_landmarks(image, results)
        
    
        cv2.imshow("OpenCv", image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

### Extract Keypoints

In [59]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33 * 4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468 * 3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21 * 3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21 * 3)
    return np.concatenate([pose, face, lh, rh])

### Setup Folders for Collection

In [60]:
DATA_PATH = os.path.join("MP_DATA")

# Actions to detect
actions = np.array(['hello', 'surprise'])

# Thirty videos worth of data
no_sequences = 30

# No of frames in a video 
sequence_length = 60

In [61]:
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

### Collect Keypoint Values for Training and Testing

In [62]:
cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)  as holistic:
    for action in actions:
        for sequence in range(no_sequences):
            for frame_num in range(sequence_length):
                

                ret, frame = cap.read()

                # make detection
                image, results = mediapipe_detection(frame, holistic)


                draw_landmarks(image, results)
                
                if frame_num == 0:
                    cv2.putText(image, 'STARTING COLLECTION', (120, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, f'Collecting frames for {action} Video Number {sequence}', (15, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    
                    cv2.imshow("OpenCv", image)
                    cv2.waitKey(2000)
                    
                else:
                    cv2.putText(image, f'Collecting frames for {action} Video Number {sequence}', (15, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.imshow("OpenCv", image)
                    
                    
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)
                
                
                    

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break

    cap.release()
    cv2.destroyAllWindows()


### Preprocess Data and Create labels and Features

In [63]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [64]:
label_map = {label:num for num, label in enumerate(actions)}

In [65]:
label_map

{'hello': 0, 'surprise': 1}

In [66]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), f"{frame_num}.npy"))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])
        

In [67]:
X = np.array(sequences)

In [68]:
X.shape

(60, 60, 1662)

In [69]:
y = to_categorical(labels).astype(int)

In [70]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

### Build and Train LSTM Neural Network

In [71]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [72]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [76]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(60,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [77]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [78]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
2/2 [==============================] - 3s 219ms/step - loss: 1.4279 - categorical_accuracy: 0.4737
Epoch 2/2000
2/2 [==============================] - 0s 212ms/step - loss: 27.4106 - categorical_accuracy: 0.5088
Epoch 3/2000
2/2 [==============================] - 0s 227ms/step - loss: 301.0250 - categorical_accuracy: 0.4737
Epoch 4/2000
2/2 [==============================] - 0s 214ms/step - loss: 134.4315 - categorical_accuracy: 0.4561
Epoch 5/2000
2/2 [==============================] - 0s 219ms/step - loss: 358.1637 - categorical_accuracy: 0.4912
Epoch 6/2000
2/2 [==============================] - 0s 219ms/step - loss: 423.7822 - categorical_accuracy: 0.5088
Epoch 7/2000
2/2 [==============================] - 0s 226ms/step - loss: 514.7994 - categorical_accuracy: 0.4737
Epoch 8/2000
2/2 [==============================] - 0s 219ms/step - loss: 826.8014 - categorical_accuracy: 0.5439
Epoch 9/2000
2/2 [==============================] - 0s 225ms/step - loss: 535.3575 - catego

Epoch 72/2000
2/2 [==============================] - 1s 232ms/step - loss: 2780.0532 - categorical_accuracy: 0.4386
Epoch 73/2000
2/2 [==============================] - 1s 237ms/step - loss: 4164.8560 - categorical_accuracy: 0.4035
Epoch 74/2000
2/2 [==============================] - 1s 235ms/step - loss: 4483.8369 - categorical_accuracy: 0.4912
Epoch 75/2000
2/2 [==============================] - 1s 228ms/step - loss: 2461.4983 - categorical_accuracy: 0.5263
Epoch 76/2000
2/2 [==============================] - 1s 231ms/step - loss: 3615.7886 - categorical_accuracy: 0.4561
Epoch 77/2000
2/2 [==============================] - 0s 229ms/step - loss: 3488.5530 - categorical_accuracy: 0.4386
Epoch 78/2000
2/2 [==============================] - 1s 234ms/step - loss: 2514.9080 - categorical_accuracy: 0.5439
Epoch 79/2000
2/2 [==============================] - 1s 228ms/step - loss: 1918.7993 - categorical_accuracy: 0.4386
Epoch 80/2000
2/2 [==============================] - 1s 229ms/step - los

2/2 [==============================] - 1s 228ms/step - loss: 951.2626 - categorical_accuracy: 0.5263
Epoch 143/2000
2/2 [==============================] - 1s 231ms/step - loss: 340.1847 - categorical_accuracy: 0.6491
Epoch 144/2000
2/2 [==============================] - 1s 229ms/step - loss: 288.8187 - categorical_accuracy: 0.4737
Epoch 145/2000
2/2 [==============================] - 1s 230ms/step - loss: 371.5469 - categorical_accuracy: 0.4386
Epoch 146/2000
2/2 [==============================] - 1s 229ms/step - loss: 599.3503 - categorical_accuracy: 0.5614
Epoch 147/2000
2/2 [==============================] - 0s 227ms/step - loss: 603.4231 - categorical_accuracy: 0.5088
Epoch 148/2000
2/2 [==============================] - 1s 232ms/step - loss: 218.0289 - categorical_accuracy: 0.5789
Epoch 149/2000
2/2 [==============================] - 1s 233ms/step - loss: 555.6313 - categorical_accuracy: 0.5439
Epoch 150/2000
2/2 [==============================] - 1s 226ms/step - loss: 231.6860 - 

2/2 [==============================] - 1s 233ms/step - loss: 1082.2571 - categorical_accuracy: 0.3860
Epoch 213/2000
2/2 [==============================] - 1s 229ms/step - loss: 975.4766 - categorical_accuracy: 0.5263
Epoch 214/2000
2/2 [==============================] - 1s 231ms/step - loss: 1126.3790 - categorical_accuracy: 0.4561
Epoch 215/2000
2/2 [==============================] - 1s 230ms/step - loss: 927.5270 - categorical_accuracy: 0.4737
Epoch 216/2000
2/2 [==============================] - 1s 229ms/step - loss: 756.9462 - categorical_accuracy: 0.5439
Epoch 217/2000
2/2 [==============================] - 1s 235ms/step - loss: 747.9999 - categorical_accuracy: 0.5789
Epoch 218/2000
2/2 [==============================] - 1s 238ms/step - loss: 986.2210 - categorical_accuracy: 0.4035
Epoch 219/2000
2/2 [==============================] - 1s 234ms/step - loss: 1139.7744 - categorical_accuracy: 0.5263
Epoch 220/2000
2/2 [==============================] - 1s 238ms/step - loss: 693.7615

2/2 [==============================] - 1s 237ms/step - loss: 9038.5654 - categorical_accuracy: 0.5088
Epoch 283/2000
2/2 [==============================] - 1s 255ms/step - loss: 6375.4282 - categorical_accuracy: 0.5263
Epoch 284/2000
2/2 [==============================] - 1s 230ms/step - loss: 8703.1064 - categorical_accuracy: 0.4035
Epoch 285/2000
2/2 [==============================] - 1s 238ms/step - loss: 7544.7720 - categorical_accuracy: 0.4035
Epoch 286/2000
2/2 [==============================] - 1s 244ms/step - loss: 2373.3530 - categorical_accuracy: 0.6491
Epoch 287/2000
2/2 [==============================] - 1s 234ms/step - loss: 4789.0806 - categorical_accuracy: 0.5088
Epoch 288/2000
2/2 [==============================] - 1s 232ms/step - loss: 2841.6199 - categorical_accuracy: 0.4912
Epoch 289/2000
2/2 [==============================] - 1s 231ms/step - loss: 3476.6091 - categorical_accuracy: 0.4386
Epoch 290/2000
2/2 [==============================] - 1s 227ms/step - loss: 419

...

2/2 [==============================] - 1s 243ms/step - loss: 642.3651 - categorical_accuracy: 0.4737
Epoch 1678/2000
2/2 [==============================] - 1s 241ms/step - loss: 364.3226 - categorical_accuracy: 0.4561
Epoch 1679/2000
2/2 [==============================] - 1s 238ms/step - loss: 1097.9569 - categorical_accuracy: 0.5263
Epoch 1680/2000
2/2 [==============================] - 1s 233ms/step - loss: 658.2126 - categorical_accuracy: 0.5263
Epoch 1681/2000
2/2 [==============================] - 1s 240ms/step - loss: 583.2910 - categorical_accuracy: 0.4737
Epoch 1682/2000
2/2 [==============================] - 1s 231ms/step - loss: 623.8305 - categorical_accuracy: 0.4737
Epoch 1683/2000
2/2 [==============================] - 1s 234ms/step - loss: 381.9239 - categorical_accuracy: 0.5263
Epoch 1684/2000
2/2 [==============================] - 1s 233ms/step - loss: 362.3533 - categorical_accuracy: 0.5263
Epoch 1685/2000
2/2 [==============================] - 1s 238ms/step - loss: 62

2/2 [==============================] - 1s 235ms/step - loss: 172.4435 - categorical_accuracy: 0.5263
Epoch 1748/2000
2/2 [==============================] - 1s 232ms/step - loss: 242.6185 - categorical_accuracy: 0.4035
Epoch 1749/2000
2/2 [==============================] - 1s 238ms/step - loss: 191.8966 - categorical_accuracy: 0.4561
Epoch 1750/2000
2/2 [==============================] - 1s 233ms/step - loss: 149.4821 - categorical_accuracy: 0.4737
Epoch 1751/2000
2/2 [==============================] - 1s 236ms/step - loss: 178.0885 - categorical_accuracy: 0.4912
Epoch 1752/2000
2/2 [==============================] - 1s 228ms/step - loss: 173.5711 - categorical_accuracy: 0.5263
Epoch 1753/2000
2/2 [==============================] - 1s 232ms/step - loss: 494.7460 - categorical_accuracy: 0.4737
Epoch 1754/2000
2/2 [==============================] - 1s 233ms/step - loss: 214.0182 - categorical_accuracy: 0.5614
Epoch 1755/2000
2/2 [==============================] - 1s 234ms/step - loss: 246

2/2 [==============================] - 1s 240ms/step - loss: 507.5102 - categorical_accuracy: 0.5263
Epoch 1818/2000
2/2 [==============================] - 1s 235ms/step - loss: 459.8610 - categorical_accuracy: 0.5263
Epoch 1819/2000
2/2 [==============================] - 1s 236ms/step - loss: 278.7123 - categorical_accuracy: 0.4737
Epoch 1820/2000
2/2 [==============================] - 1s 242ms/step - loss: 191.1651 - categorical_accuracy: 0.5088
Epoch 1821/2000
2/2 [==============================] - 1s 243ms/step - loss: 128.1299 - categorical_accuracy: 0.4386
Epoch 1822/2000
2/2 [==============================] - 1s 240ms/step - loss: 97.8996 - categorical_accuracy: 0.4386
Epoch 1823/2000
2/2 [==============================] - 1s 236ms/step - loss: 86.3027 - categorical_accuracy: 0.5263
Epoch 1824/2000
2/2 [==============================] - 1s 241ms/step - loss: 334.2843 - categorical_accuracy: 0.4737
Epoch 1825/2000
2/2 [==============================] - 1s 243ms/step - loss: 345.0

Epoch 1887/2000
2/2 [==============================] - 1s 236ms/step - loss: 2247.1182 - categorical_accuracy: 0.4737
Epoch 1888/2000
2/2 [==============================] - 1s 231ms/step - loss: 1075.2736 - categorical_accuracy: 0.5263
Epoch 1889/2000
2/2 [==============================] - 1s 235ms/step - loss: 941.5795 - categorical_accuracy: 0.5088
Epoch 1890/2000
2/2 [==============================] - 1s 232ms/step - loss: 602.5222 - categorical_accuracy: 0.5439
Epoch 1891/2000
2/2 [==============================] - 1s 237ms/step - loss: 745.6033 - categorical_accuracy: 0.4211
Epoch 1892/2000
2/2 [==============================] - 1s 249ms/step - loss: 761.8339 - categorical_accuracy: 0.5614
Epoch 1893/2000
2/2 [==============================] - 1s 237ms/step - loss: 713.1266 - categorical_accuracy: 0.5263
Epoch 1894/2000
2/2 [==============================] - 1s 243ms/step - loss: 2346.5461 - categorical_accuracy: 0.4737
Epoch 1895/2000
2/2 [==============================] - 1s 233

2/2 [==============================] - 1s 233ms/step - loss: 766.5935 - categorical_accuracy: 0.4737
Epoch 1957/2000
2/2 [==============================] - 1s 235ms/step - loss: 1110.8855 - categorical_accuracy: 0.5263
Epoch 1958/2000
2/2 [==============================] - 1s 231ms/step - loss: 877.6407 - categorical_accuracy: 0.5263
Epoch 1959/2000
2/2 [==============================] - 1s 244ms/step - loss: 1396.2346 - categorical_accuracy: 0.4737
Epoch 1960/2000
2/2 [==============================] - 1s 232ms/step - loss: 889.6157 - categorical_accuracy: 0.4737
Epoch 1961/2000
2/2 [==============================] - 1s 229ms/step - loss: 1627.8761 - categorical_accuracy: 0.5263
Epoch 1962/2000
2/2 [==============================] - 1s 234ms/step - loss: 1980.2770 - categorical_accuracy: 0.5263
Epoch 1963/2000
2/2 [==============================] - 1s 234ms/step - loss: 737.8378 - categorical_accuracy: 0.5439
Epoch 1964/2000
2/2 [==============================] - 1s 229ms/step - loss:

In [48]:
model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 30, 64)            442112    
                                                                 
 lstm_7 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_8 (LSTM)               (None, 64)                49408     
                                                                 
 dense_6 (Dense)             (None, 64)                4160      
                                                                 
 dense_7 (Dense)             (None, 32)                2080      
                                                                 
 dense_8 (Dense)             (None, 3)                 99        
                                                                 
Total params: 596,675
Trainable params: 596,675
Non-tr

In [ ]:
res = model.predict(X_test)[0]

In [ ]:
res

array([6.4898854e-19, 2.3380089e-09, 1.0000000e+00], dtype=float32)

In [ ]:
actions[np.argmax(res)]

'hello'

In [ ]:
y_test

array([[0, 0, 1],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0]])

In [ ]:
model.save('action.h5')

In [25]:
model.load_weights('action.h5')

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [ ]:
yhat = model.predict(X_test)

In [ ]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [ ]:
ytrue

[2, 0, 0, 0, 0]

In [ ]:
yhat

[2, 0, 0, 0, 0]

In [ ]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[1, 0],
        [0, 4]],

       [[4, 0],
        [0, 1]]])

In [ ]:
accuracy_score(ytrue, yhat)

1.0

### Test in real time

In [26]:
cap = cv2.VideoCapture(0)

sequence = []
sentence = []
threshold = 0.4

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)  as holistic:
    while cap.isOpened():

        ret, frame = cap.read()
        
        # make detection
        image, results = mediapipe_detection(frame, holistic)
        

        draw_landmarks(image, results)
        
        
        # prediction login
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]

        text = ""
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0 ]
            # print(actions[np.argmax(res)])
        
        
        #3. Viz logic
            if res[np.argmax(res)] > threshold: 
                text = actions[np.argmax(res)]
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            # image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, text, (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    
        cv2.imshow("OpenCv", image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

1/1 [==============================] - 0s 19ms/step


1/1 [==============================] - 0s 18ms/step


1/1 [==============================] - 0s 20ms/step


1/1 [==============================] - 0s 18ms/step


1/1 [==============================] - 0s 17ms/step


1/1 [==============================] - 0s 22ms/step


KeyboardInterrupt: 

Selenium To Search in Google

In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import time
#set chromodriver.exe path
driver = webdriver.Chrome(ChromeDriverManager().install())
# driver = webdriver.Chrome(executable_path="C:\chromedriver.exe")
driver.implicitly_wait(0.5)
#launch URL
driver.get("https://www.google.com/")
#identify search box
m = driver.find_element_by_name("q")
#enter search text
m.send_keys("Query Goes Here")
time.sleep(0.2)
#perform Google search with Keys.ENTER
m.send_keys(Keys.ENTER)